In [ ]:
# Testing Cell
from aviary.mission.flight_phase_builder import FlightPhaseOptions
from aviary.utils.doctape import glue_variable

opts = FlightPhaseOptions()
for key in opts:
    glue_variable(key, md_code=True)

# The `phase_info` Format

## Overview

Aviary uses a nested python dictionary to define and control all aspects of the mission and its phases. Understanding this format is necessary if you want to modify existing missions or design your own.

The top level of the phase_info contains a name string and a sub-dictionary for each phase in the mission. Users are free to choose any valid python name for their phases (valid characters are upper/lower case letters, numbers, and underscores.) In addition, there are two default phases "pre_mission" and "post_mission". These contain any settings that pertain to the pre_mission, which is any calculation that occurs before the mission runs, and the post_mission, which is any calculation that occurs after the mission runs.


In [ ]:
phase_info = {
    'pre_mission': {},
    'climb1': {},
    'climb2': {},
    'cruise': {},
    'descent': {},
    'post_mission': {},
}

Note that the order is important, so 'pre_mission' should be first, followed by phases in chronological order, terminated by 'post_mission'.

## Phase Options

Each phase has its own options that are organized like this:


In [ ]:
phase_info['climb1'] = {
    'subsystem_options': {},
    'user_options': {},
    'initial_conditions': {},
    'external_subsystems': [],
}

The 'subsystem_options' dictionary contains settings for core subsystems, primarily the core aerodynamics calculations.
The 'user_options' dictionary includes all settings that define the phase and its boundary conditions.
The 'initial_conditions' dictionary allows you to set initial values for any variables thate are under control of the optimizer, including states, optimized controls, and time.
Finally, 'external_subsystems' is a list that contains all of the external builder objects.

## Subsystem Options

The subsystem options for core aerodynamics are outlined in [Externally Computed Polars](external_aero.ipynb).

## User Options

An overview of user options for a Height Energy phase is given in the [Level 1 Introduction](../getting_started/onboarding_level1.ipynb), and an example for a cruise phase is shown in the [Level 2 Introduction](../getting_started/onboarding_level2.ipynb). These are a good starting points for understanding how these options are specified. The Aviary trajectory is constructed using phase builders. In general, each option is a name, value pair where the names are intended to follow a specific pattern. In particular, when the option name includes a variable name, that name goes first. For example, the initial and final values for constraints on altitude would be "altitude_initial" and "altitude_final". Each value could be a number, tuple/list, array, or a boolean as needed. The user options format supports (and enforces) the inclusion of an OpenMDAO unit string when specifying any quantity that has a unit. These take the format "(500, 's')". If an option requires units and takes multiple value, the format "((300.0, 900.0), 's')" is used.  Some variables (such as Mach) require an explicit "unitless" as the unit.

Each builder has its own options that are tied to the equations of motion that it models. Thus, since the two-degree-of-freedom (2DOF) equations have different states and controls than the height energy equations, the phase_infos have some user options that are different. The following section describes the general patterns that apply to the phases that are included in Aviary. Note that, at present, the height energy phases fully adhere to this standard, while the '2DOF' phases are still being converted.

###General phase settings

- **{glue:md}num_segments**: The number of segments in the dymos transcription.

- **{glue:md}order**: The order of polynomials for the dymos transcription.

- **{glue:md}throttle_enforcement**: Enforce throttle as a path_constraint (default), boundary_constraint, a control (which can be optimized or set to a fixed value), or a solver bound (not recommended).

- **{glue:md}throttle_allocation**: How to allocate throttles for multi-engine, can be [‘fixed’, ‘static’, ‘dynamic’].

- **{glue:md}no_climb**: Set to True to prevent the aircraft from climbing during the phase. This option can be used to prevent unexpected climb during a descent phase.

- **{glue:md}no_descent**: Set to True to prevent the aircraft from descending during the phase. This can be used to prevent unexpected descent during a climb phase.

###Specialized phase settings

- **{glue:md}reserve**: When True, this is a reserve phase. These phases should be after your flight phases.

- **{glue:md}ground_roll**: When True, restrict the problem to the ground plane. This will remove certain states/controls, and is only used in takeoff and landing phases.

- **{glue:md}required_available_climb_rate**: Adds a lower bound path constraint on altitude_rate_max, which is the maximum rate that the aircraft could climb at any point in the phase.

- **{glue:md}target_distance**: Total mission distance constraint. This should be set in the final flight phase only.

###State variables

In a height energy phase, the states are mass and distance. In a two degree of freedom phase, the states also include altitude and velocity.

- **{glue:md}mass_initial**: Mass at the start of the phase. When unspecified, the optimizer controls the value. When specified, a constraint is created on the initial mass.

- **{glue:md}mass_final**: Mass at the end of the phase. When unspecified, the optimizer controls the value. When specified, a constraint is created on the final mass.

- **{glue:md}mass_bounds**: Tuple of upper and lower bounds for all values of mass in the phase. The default of None for either means that bound will not be declared.

- **{glue:md}mass_ref**: Multiplicative scale factor "ref" for mass. Default is 1.0

- **{glue:md}mass_ref0**: Additive scale factor "ref0" for mass. Default is None.

- **{glue:md}mass_defect_ref**: Multiplicative scale factor "ref" for defect constraint. Deafult is None, which means the ref and defect_ref are the same.

- **{glue:md}mass_solve_segments**: When True, a solver will be used to converge the mass collocation defects within a segment. Note that the state continuity defects between segements will still be handled by the optimizer.

###Controls

In a height energy phase, the controls are mach and altitude. In a two degree of freedom phase, the controls are throttle or angle of attack depending on the phase.

- **{glue:md}altitude_optimize**: When True, the optimizer will set this value. When False, the initial value for all nodes can be set in the initial_conditions section of the phase.

- **{glue:md}altitude_initial**: Altitude at the start of the phase. If altitude_optimize is True, specifying this will create a constraint. Otherwise, it serves as an initial value for the first point. When unspecified, the linkage constraint with the upstream phase will drive this value if altitude_optimize is True.

- **{glue:md}altitude_final**: Altitude at the end of the phase. If altitude_optimize is True, specifying this will create a constraint. Otherwise, it serves as an initial value for the first point. When unspecified, the linkage constraint with the downstream phase will drive this value if altitude_optimize is True.

- **{glue:md}altitude_bounds**: Tuple of upper and lower bounds for all values of altitude in the phase. The default of None for either means that bound will not be declared.

- **{glue:md}altitude_ref**: Multiplicative scale factor "ref" for altitude. Default is 1.0

- **{glue:md}altitude_ref0**: Additive scale factor "ref0" for altitude. Default is None.

- **{glue:md}altitude_polynomial_order**: The order of polynomials for interpolation in the transcription. Default is None, which does not use a polynomial.

###Time
- **{glue:md}time_initial**: Value of "time" at the start of the phase. When unspecified, the value is determined by the optimizer.

- **{glue:md}time_duration**: Duration of the phase. When unspecified, the value is determined by the optimizer.

- **{glue:md}time_initial_bounds**: Upper and lower bounds for time_initial. Ignored if "time_initial" is specified. 

- **{glue:md}time_duration_bounds**: Upper and lower bounds for time_duration. 

- **{glue:md}time_initial_ref**: Multiplicative scale factor "ref" for time_initial. Default is 1.0

- **{glue:md}time_duration_ref**: Additive scale factor "ref" for time_duration. Default is None.

A full description of every user option in the phase_info for each of Aviary's phase builders is given in the [Complete phase_info Reference.](phase_info_detailed.ipynb).

## Pre and Post mission Options
TODO: Coming Soon.